In [6]:
"""
Script for generating excelfiles of rates, (adjusted or unadjusted) OR, by disease, datatype, and year:
In: 
    - Disease specific SAS files, e.g. "gbddis_X_X.sas7bdat"
Out: 
    - Disease specific files for PC or NBHW, e.g. "PC_outcome_4_DISX_by_DispInkFamLag5_1_r_X.xlsx" or "All_outcome_4_DISX_by_DispInkFamLag5_1_r_X.xlsx"


By Pär, June 2023 (precursor, CreateTimeseriesGBDdis_ses_2beRunInParallel_june23)
"""

'\nScript for generating excelfiles of rates, (adjusted or unadjusted) OR, by disease, datatype, and year:\nIn: \n    - Disease specific SAS files, e.g. "gbddis_X_X.sas7bdat"\nOut: \n    - Disease specific files for PC or NBHW, e.g. "PC_outcome_4_DISX_by_DispInkFamLag5_1_r_X.xlsx" or "All_outcome_4_DISX_by_DispInkFamLag5_1_r_X.xlsx"\n\n\nBy Pär, June 2023\n'

In [1]:
#pip install pyreadstat

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import pyreadstat
import patsy
import pyreadstat
import statsmodels.formula.api as smf
import datetime as dt
from datetime import datetime
import time # for sleep
%matplotlib inline

In [2]:
# Some functions
def wrap_error(func):
    """
    To repress error outputs
    """
    def func_wrapper(*args, **kwargs):
        try:
           return func(*args, **kwargs)
        except:
           pass
    return func_wrapper


@wrap_error
def odds_r_calculation(df, expr):
    """
    -Input: 
        -df
        -expr, (Patsy)
    -Out: 
        OR
        OR_CI_L
        OR_CI_U
    """
    
    log_reg = smf.logit(expr, data=df ).fit()
    
    odds_ratios = pd.DataFrame(
    {
        "OR": log_reg.params,
        "LowerCI": log_reg.conf_int()[0],
        "UpperCI": log_reg.conf_int()[1],
        
    })
    
    odds_ratios = np.exp(odds_ratios) # make OR
    
    index_OR_of_interrest = 1 # model is specified with intercept at 0 and effect of interest at 1

    odds_ratios_lowByHigh = odds_ratios.iloc[index_OR_of_interrest,:]
    odds_ratios_lowByHigh['pvalue'] = log_reg.pvalues.iloc[index_OR_of_interrest]
    
    return odds_ratios_lowByHigh

In [3]:
def send_ping(DisNr,AR,add = 'begining'):
    # for keeping track and keeping serverconnection
    ts = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
    text2write = "DisNr_%s_ForYear_%s_written \n%s punkt. %s" %(DisNr,AR,ts,add)
    
    fp_ping1 = r'C:\x\pingFileF_ses.txt'
    fp_ping2 = r'C:\x\pingFileF_ses.txt'

    file1 = open(fp_ping1, "+r") #"+r") # first time 'w'
    file1.write(text2write )
    file1.close()

    file2 = open(fp_ping2, "+r") #"+r") 
    file2.write(text2write )
    file2.close()


In [4]:
#call to start
send_ping('1',1,add = 'begining')

In [12]:
#Wait 3 h
#time.sleep(10800) 

#Wait 4 h
#time.sleep(14400) 


In [1]:
%%time
# sök för ChangeBack mars 22 2023 och återställ när vi vet skript funakr

# OBS makes sure correct disease (not exluding 15 e.g.)

dtchangedic = {"LopNr":'int32',
               "FodelseLandG":'category',
               "DispInkFamLag5_1":'float32',
               "DispInkFamLag5_1_r":'float32',
               #'Lan':'category', # categorical works when not using Lan as predictor, not otherwise
               #"Civil":'category',
               "Sun2000niva":'int32',
               "PC_u":'float32',
               "PC_e":'float32',
               "SO_u":'float32',
               "SO_e":'float32',
               "SI_u":'float32',
               "SI_e":'float32',
               'Death':'float32'}



sns.set_style("whitegrid", {'xtick.bottom': True})
fz = (20, 8) # w, h

resultDicAllDisAllYrsAllOutc ={}
resultDicAllDisAllYrsPC      ={}


# Specify SES
SESvarName ="DispInkFamLag5_1_r"
SESvarNamePrecursor = 'DispInkFamLag5_1_r'
SESlow = 0 #"1"#'A' # A =rural, B eller C =centralort
SEShigh =4 #"2" #'C'


# sepficy models to run over (m0-m2), and provide sufix for outpuytfiles
modelDicName ={'m0': 'unadjusted' ,'m1': 'adjNoCounty', 'm2':'adjCounty'}


startYear= 2004
print(startYear)

gbdNr2namedic ={8:"Neoplasm",9:'Cardiovascular',19:'Musculoskeletal',13:'Mental',12:'Neurological' , 
                10:'Chronic_respiratory' , 15: 'Diabetes', 23: 'Self-harm_violence',100: 'All'}



for modelNr in ['m1','m0', 'm2']:
    for DisNr in [100, 9, 8 , 12,10,15]:# [10,15,23,9, 8 , 12]: #[9, 8 , 12, 13,19]
        #ChangeBack mars 22 2023
        for AR in range(startYear,2018): #2020): [2017]:#
            print('TRACK: DispInkFamLag5_1_r, Initierar DisNr ', DisNr,' //T:',datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


            #load data
            fp = C:\x'\gbddis_%s_%s.sas7bdat' %(DisNr, AR)

            df, meta = pyreadstat.read_sas7bdat(fp, encoding="LATIN1")
            send_ping(DisNr,AR,add = 'loaded first')

            print('TRACK: DisNr ', DisNr,' for ',AR,' was loaded, //T:',datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

            # CLEAN data frame from rows with missing values of essential variables
            df = df[df[SESvarNamePrecursor].notna()] # keep only valid 

            #df = df[df.Desoletter.isin(['A', 'B', 'C'])]
            df = df[df.LopNr>0] # only lopnr largrer than 0 (some counties provided wrong lopnr)
            df = df[df[SESvarName].notna()] # only include rows where we have proper income rank
            df = df[df.Kon.isin(["1","2"])]
            df.replace([np.inf, -np.inf], np.nan, inplace=True)
            df = df.fillna(0)

            df['Sun2000niva'] = pd.to_numeric(df['Sun2000niva'],errors='coerce') # creates nan for invalid edu
            df = df[df["Sun2000niva"].notna()] # remove invalid edu

            #only include proper Län /needed for regression
            df = df[df.Lan.isin(['01', '03', '04','05', '06', '07',  '08', '09','10','12','13','14','17','18','19','20','21','22','23','24','25'])]
            df['FoddAr'] = pd.to_numeric(df['FoddAr'],errors='coerce') # creates nan for invalid FoddAr
            df = df[df["FoddAr"].notna()] # removes rows with missing FoddAr

            # for LISA > 2010, also 15 yrs old are included. Make konsekvent, remove all 15 and below. 
            df['Age'] = AR-df['FoddAr']
            df = df[df['Age']>15]
            df['AgeP2'] = df['Age']**2     # non-linear age effects
            # Binarize Civil varialber, (for fewer and an less coliniear regressors))
            df['CivStat'] = 0              # default Civilst ej gift
            df.loc[df['Civil'] == 'G','CivStat' ] = 1 # 1 if gift. 
            df.drop(columns="Civil", inplace=True)
            print('BEFORE CHANGED DTYPE', df.info())
            df = df.astype(dtchangedic, copy=False, errors='raise') #dtchangedic specified above // copy=True,

            # fixa PC_U_a  
            df['PC_U_a']= df['PC_u'] - df['SO_u'] 
            df.loc[df['PC_U_a']<0, "PC_U_a"]= 0  #remove neg


            # create PC denominator
            ar = AR
            df['PC_denom'] = df['Lan'].apply(lambda i: 1 if (          (i =='01' and ar>= 2004) or  
                                                                       (i =='03' and ar>= 2010) or
                                                                       (i =='05' and ar>= 1999)  or
                                                                       (i =='06' and ar>= 2012)  or
                                                                       (i =='08' and ar>= 2011)  or
                                                                       (i =='09' and ar>= 2013)  or
                                                                       (i =='10' and ar>= 2010)  or
                                                                       (i =='12' and ar>= 2004)  or
                                                                       (i =='14' and ar>= 2000)  or
                                                                       (i =='17' and ar>= 2014)  or
                                                                       (i =='18' and ar>= 2002)  or
                                                                       (i =='19' and ar>= 2017)  or
                                                                       (i =='20' and ar>= 2005)  or
                                                                       (i =='21' and ar>= 2010)  or
                                                                       (i =='22' and ar>= 2011)  or
                                                                       (i =='23' and ar>= 2010)  or
                                                                       (i =='25' and ar>= 2012)
                                                                        )                                         
                                                            else 0)  


            # prepp for aggregate
            df.rename(columns={"FoddAr":"PopInDemog"}, inplace =True)     # just to count number of people

            # for "per 100.000" - 
            df_gb = df.groupby(SESvarName).agg({'PC_u': ['sum'],  # here, look at Inkome as Q
                  'PC_e': ['sum'],
                  'PC_U_a': ['sum'],                                      
                  'SO_u': ['sum'],
                  'SO_e': ['sum'],
                  'SI_u': ['sum'],
                  'SI_e': ['sum'],
                  'Death': ['sum'],
                  'PC_denom': ['sum'],                                
                  'PopInDemog': ['count']} )

            # insert total population in each demogro
            gb_iq = df_gb.groupby(by= SESvarName).sum()
            PopSumByGroupS = gb_iq.sum(axis=0)
            PopSumByGroupS.name =  99 #Tot population name
            gb_iq = gb_iq.append(PopSumByGroupS)
            gb_iq.columns = gb_iq.columns.droplevel(level=1)

            #differentiate between NPR (national) denominator and PC
            gb_iq_d=gb_iq.div(gb_iq["PopInDemog"],axis=0)*100000
            gb_iq_pc=gb_iq[["PC_u","PC_e","PC_U_a"]].div(gb_iq["PC_denom"],axis=0).replace(np.inf, 0).fillna(0)*100000 #replace(np.inf, 0) to nullify when no PC data

            # ratio low by high (#_low/100k by #_high/100k), SESlow = 0, SEShigh=4
            low_div_high = gb_iq_d.loc[SESlow,:].div(gb_iq_d.loc[SEShigh,:],fill_value=0).fillna(0)
            low_div_high.name =  55 # low vs high
            gb_iq_d = gb_iq_d.append(low_div_high)

            low_div_high = gb_iq_pc.loc[SESlow,:].div(gb_iq_pc.loc[SEShigh,:],fill_value=0).fillna(0)
            low_div_high.name =  55 
            gb_iq_pc = gb_iq_pc.append(low_div_high)


            gb_iq_d["PopInDemog"]  = gb_iq["PopInDemog"]
            gb_iq_pc["PC_denom"]      = gb_iq["PC_denom"]

            gb_iq_d['ar'] = AR
            gb_iq_pc['ar']= AR

            ########################

            outcomeList = ["PC_u", "SO_u", "SI_u", "Death"]

            for VarOfInt in outcomeList:

                designM_raw  = "%s ~  C(DispInkFamLag5_1_r, Treatment(reference=4))" %(VarOfInt) # consider including county
                designM_adj1 = "%s ~  C(DispInkFamLag5_1_r, Treatment(reference=4)) + Age + AgeP2 + C(CivStat) + C(FodelseLandG) + C(Kon)" %(VarOfInt)
                designM_adj2 = "%s ~  C(DispInkFamLag5_1_r, Treatment(reference=4)) + C(Lan) + Age + AgeP2 + C(CivStat) + C(FodelseLandG) + C(Kon)" %(VarOfInt) 

                modelDic ={'m0': designM_raw ,'m1': designM_adj1, 'm2':designM_adj2}
                designM = modelDic[modelNr]

                """ 
                #designM = "%s ~  C(DispInkFamLag5_1_r, Treatment(reference=4))  + Age + AgeP2 + C(CivStat) + C(FodelseLandG) + C(Kon)" %(VarOfInt) # consider including county
                #designM = "%s ~ C(Kon,Treatment(reference=1))  + C(EDU3) + Age + AgeP2 + C(CivStat) + C(FodelseLandG) + C(DispInkFamLag5_1_r)" %(VarOfInt) #+ C(DispInkFamLag5_1_r)
                #designM = "%s ~ C(EDU3 ,Treatment(reference=3)) + C(Kon)  + Age + AgeP2 + C(CivStat) + C(FodelseLandG) " %(VarOfInt) #+ C(DispInkFamLag5_1_r)
                #designM = "%s ~ C(Desoletter ,Treatment(reference='C')) + C(Kon)  + Age + AgeP2 + C(CivStat) + C(FodelseLandG) + C(DispInkFamLag5_1_r)" %(VarOfInt)
                """
                print('TRACK: Initierar OR calc för', VarOfInt, ' för ', AR , ' för ', DisNr,'// T:',datetime.now().strftime("%d/%m/%Y %H:%M:%S"))

                OR   = np.nan
                OR_L = np.nan
                OR_U = np.nan
                OR_p = np.nan 

                if  AR in range(2000, 2020):
                    # if VarOfInt = PC_u
                    if VarOfInt == "PC_u": 
                        df2OR = df[df['PC_denom']==1].copy()
                    else: 
                        df2OR = df.copy()

                    or_results = odds_r_calculation(df2OR, designM)  # odds_r_calculation takes first specified contrast () and return os results

                    try: #The truth value of a Series is ambiguous. //if or_results!= None:
                        or_results[np.isinf(or_results)] = 0 # clear any inf values
                        OR   = or_results['OR']
                        OR_L = or_results['LowerCI']
                        OR_U = or_results['UpperCI']
                        OR_p = or_results['pvalue']
                    except:
                        OR   = 0
                        OR_L = 0
                        OR_U = 0 
                        OR_p = 1

                #feed in OR
                #var names
                OR_M_VarOfInt = "OR_M_%s"  %(VarOfInt)
                OR_L_VarOfInt = "OR_L_%s"  %(VarOfInt)
                OR_U_VarOfInt = "OR_U_%s"  %(VarOfInt)
                OR_p_VarOfInt = "OR_p_%s"  %(VarOfInt)
                #var values
                gb_iq_d[OR_M_VarOfInt] = OR
                gb_iq_d[OR_L_VarOfInt] = OR_L
                gb_iq_d[OR_U_VarOfInt] = OR_U
                gb_iq_d[OR_p_VarOfInt] = OR_p
                gb_iq_pc[OR_M_VarOfInt] = OR
                gb_iq_pc[OR_L_VarOfInt] = OR_L
                gb_iq_pc[OR_U_VarOfInt] = OR_U
                gb_iq_pc[OR_p_VarOfInt] = OR_p

            #build gb_iq_d_allY , gb_iq_pc_allY
            if AR == startYear: # instantiate new dataframe
                gb_iq_d_allY  = gb_iq_d
                gb_iq_pc_allY = gb_iq_pc
            else: 
                gb_iq_d_allY  = gb_iq_d_allY.append(gb_iq_d)   
                gb_iq_pc_allY = gb_iq_pc_allY.append(gb_iq_pc)

            del df, meta, df2OR, gb_iq

        print('TRACK: Tom slutet av', AR, ' för ', DisNr,' //T:',datetime.now().strftime("%d/%m/%Y %H:%M:%S"))


        print('TRACK: Tom börja bygga gb_iq_d_allY_gb',' //T:',datetime.now().strftime("%d/%m/%Y %H:%M:%S"))  
        gb_iq_d_allY_gb  = gb_iq_d_allY.groupby(by = ['ar',SESvarName]).mean()
        gb_iq_pc_allY_gb = gb_iq_pc_allY.groupby(by = ['ar',SESvarName]).mean() 

        # Save
        
        # NBHW
        df2pl_all = gb_iq_d_allY_gb.groupby(by = ['ar',SESvarName]).mean()
        dicKeyDis = "%s" %(DisNr)
        resultDicAllDisAllYrsAllOutc[dicKeyDis]= df2pl_all
        excelName2save = "All_outcome_4_%s_by_%s_%s.xlsx" %(gbdNr2namedic[DisNr],SESvarName, modelDicName[modelNr])
        df2pl_all.to_excel(excelName2save)


        # PC 
        df2pl_pc = gb_iq_pc_allY_gb.groupby(by = ['ar',SESvarName]).mean()
        resultDicAllDisAllYrsPC[dicKeyDis]= df2pl_pc
        excelName2save = "PC_outcome_4_%s_by_%s_%s.xlsx" %(gbdNr2namedic[DisNr],SESvarName, modelDicName[modelNr])
        df2pl_pc.to_excel(excelName2save)
        
        print('TRACK: Sparat ', excelName2save, ' //T:',datetime.now().strftime("%d/%m/%Y %H:%M:%S")) 
        #send_ping('Sparat  df2pl_pc')






NameError: name 'sns' is not defined

In [ ]:
df

In [ ]:
===== junk below==========

In [15]:
db = pd.DataFrame(data ={'a':[1,3,4], 'b':['a',2,4]})
dtchange = {"a":'int32','b':'int32'}
db.astype(dtchange, errors='ignore')

,a,b
0,1,a
1,3,2
2,4,4


In [21]:
db['b'] = pd.to_numeric(db['b'],errors='coerce')


In [35]:
a= 1
a=3
a =

SyntaxError: invalid syntax (Temp/ipykernel_20208/3149397849.py, line 3)

In [12]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7245308 entries, 0 to 7512244
Data columns (total 22 columns):
 #   Column              Dtype  
---  ------              -----  
 0   LopNr               float64
 1   FoddAr              int32  
 2   Kon                 object 
 3   DodDatum            object 
 4   FodelseLandG        float64
 5   Lan                 object 
 6   Sun2000niva         object 
 7   ar                  float64
 8   Desoletter          object 
 9   DispInkKE           float64
 10  DispInkFamLag5_1    float64
 11  DispInkFamLag5_1_r  float64
 12  PC_u                float64
 13  PC_e                float64
 14  SO_u                float64
 15  SO_e                float64
 16  SI_u                float64
 17  SI_e                float64
 18  Death               float64
 19  Age                 int32  
 20  AgeP2               int32  
 21  CivStat             int64  
dtypes: float64(13), int32(3), int64(1), object(5)
memory usage: 1.2+ GB


In [ ]:
def gini(x):
    total = 0
    for i, xi in enumerate(x[:-1], 1):
        total += np.sum(np.abs(xi - x[i:]))
    return total / (len(x)**2 * np.mean(x))

In [38]:
gb_iq_pc_allY

,PC_u,PC_e,PC_U_a,PC_denom,ar,OR_M_PC_u,OR_L_PC_u,OR_U_PC_u,OR_M_SO_u,OR_L_SO_u,OR_U_SO_u,OR_M_SI_u,OR_L_SI_u,OR_U_SI_u,OR_M_Death,OR_L_Death,OR_U_Death
DispInkFamLag5_1_r,,,,,,,,,,,,,,,,,
0.0,884.894276,907.388785,875.301029,302296.0,2000,1.580652,1.480738,1.687308,1.355532,1.237469,1.484859,1.467502,1.400999,1.537163,1.111285,1.015164,1.216509
1.0,894.301145,917.783165,887.686492,302359.0,2000,1.580652,1.480738,1.687308,1.355532,1.237469,1.484859,1.467502,1.400999,1.537163,1.111285,1.015164,1.216509
2.0,865.537600,886.576113,856.990704,304204.0,2000,1.580652,1.480738,1.687308,1.355532,1.237469,1.484859,1.467502,1.400999,1.537163,1.111285,1.015164,1.216509
3.0,814.272553,832.763739,807.668559,302847.0,2000,1.580652,1.480738,1.687308,1.355532,1.237469,1.484859,1.467502,1.400999,1.537163,1.111285,1.015164,1.216509
4.0,667.927399,684.634471,663.661764,281318.0,2000,1.580652,1.480738,1.687308,1.355532,1.237469,1.484859,1.467502,1.400999,1.537163,1.111285,1.015164,1.216509
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2.0,2815.271750,3978.091161,2174.304149,850433.0,2008,0.000000,0.000000,0.000000,1.151144,1.128343,1.174407,1.778320,1.696356,1.864244,1.986320,1.743087,2.263494
3.0,2609.201352,3690.078103,2004.850163,859765.0,2008,0.000000,0.000000,0.000000,1.151144,1.128343,1.174407,1.778320,1.696356,1.864244,1.986320,1.743087,2.263494
4.0,2337.416089,3491.291649,1712.310962,980955.0,2008,0.000000,0.000000,0.000000,1.151144,1.128343,1.174407,1.778320,1.696356,1.864244,1.986320,1.743087,2.263494


In [15]:
df.head()

,LopNr,PopInDemog,Kon,FodelseLandG,Lan,Sun2000niva,ar,Desoletter,DispInkFamLag5_1,DispInkFamLag5_1_r,...,SO_e,SI_u,SI_e,Death,EDU3,Age,AgeP2,CivStat,PC_U_a,PC_denom
0,2,1961,2,1.0,01,547,2019.0,C,4230.399902,2.0,...,0.0,0.0,0.0,0.0,3,58,3364,0,0.0,1
1,7,1993,2,1.0,01,323,2019.0,C,4462.200195,3.0,...,1.0,0.0,0.0,0.0,2,26,676,0,0.0,1
2,9,1969,1,1.0,13,317,2019.0,B,2355.600098,0.0,...,0.0,0.0,0.0,0.0,2,50,2500,0,0.0,0
3,11,1950,1,3.0,01,200,2019.0,C,1690.800049,0.0,...,0.0,0.0,0.0,0.0,1,69,4761,0,0.0,1
4,13,1998,1,1.0,12,522,2019.0,C,2549.600098,0.0,...,0.0,0.0,0.0,0.0,3,21,441,0,0.0,1


In [11]:
set(df.Kon)

{'1', '2'}

In [9]:
df, meta = pyreadstat.read_sas7bdat(fp, encoding="LATIN1")


In [10]:
df.head()

,LopNr,FoddAr,Kon,FodelseLandG,Civil,Lan,Sun2000niva,ar,Desoletter,DispInkFamLag5_1,DispInkFamLag5_1_r,PC_u,PC_e,SO_u,SO_e,SI_u,SI_e,Death
0,2.0,1961,2,1.0,G,01,547,2000.0,C,3798.4,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,4.0,1932,2,2.0,G,08,999,2000.0,C,790.8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,9.0,1969,1,1.0,OG,12,317,2000.0,C,1004.8,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,10.0,1927,1,1.0,S,21,547,2000.0,C,2703.8,4.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11.0,1950,1,3.0,G,18,200,2000.0,C,2230.0,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
